In [1]:
import gym
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# load data
data_dict = {
    "fleet_path":'data/fleet_small.csv',
    "ports_path":'data/ports_10.csv',
    "dm_path" : 'data/distance_matrix.csv'
}


ports_df = pd.read_csv(data_dict['ports_path'])
fleet_df = pd.read_csv(data_dict['fleet_path'])
distance_matrix = pd.read_csv(data_dict['dm_path'])




In [ ]:
from env.env import CarbonEnv

In [ ]:
# Delay

contracts = create_contrs(seed=56)
agent_speed = 10
agent_ship = 2
ship_current_port = 4
contract = contracts.loc[0]
start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_hours = find_distance(ship_current_port,start_port_number) / agent_speed
print(dt_1_hours)
trip_2_hours = find_distance(start_port_number,end_port_number) / agent_speed
print(dt_2_hours)
trip_total_hours = trip_1_hours + trip_2_hours


trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")

In [ ]:
# cargo_size vs ship_shize

contracts = create_contrs(seed=56)

# gia ka8e contract se spawned contracts

contract = contracts.loc[0]

fleet_df
# elegxw ka8e ploio pou einai available dhladh exei availability 1 
mask_of_available_ships = fleet_df['ship_availability'] == 1

available_ships = fleet_df[mask_of_available_ships]

available_ships_numbers = available_ships.ship_number

available_ships_numbers

In [ ]:


# checkare sto masking an kaneis mask out tous assous h ta mhdenika

# gia ka8e tetoio ploio


# get ship_number

ship_numbers = 
agent_ship_selection = 2

# get ship info
ship_mask = fleet_df['ship_number'] == agent_ship_selection

ship_info = fleet_df[ship_mask]
print(ship_info)

ship_current_port = ship_info.current_port
print(ship_current_port)






start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_distance = find_distance(ship_current_port,start_port_number)
print(trip_1_distance)
trip_2_distance = find_distance(start_port_number,end_port_number)
print(trip_2_distance)
trip_total_distance = trip_1_distance + trip_2_distance




trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")


In [3]:
import gym
from gym import spaces
import pandas as pd
import numpy as np
import tensorflow as tf
from utils.utils import cii_expected


class CarbonEnv2(gym.Env):
    """
    Description :

    A custom openai gym environment for the carbon emission problem.

    """

    def __init__(self, data_dict={"fleet_path": 'data/fleet_small.csv',
                                  "ports_path": 'data/ports_10.csv',
                                  "dm_path": 'data/distance_matrix.csv'}):
        super().__init__()

        self.data_dict = data_dict

        # get fleet info in df
        self.fleet = pd.read_csv(self.data_dict['fleet_path'])

        # get port info in df
        ports = pd.read_csv(self.data_dict['ports_path'])
        self.ports = ports.loc[:, ['number', 'name', 'country']]

        # get distance matrix info
        self.dm_df = pd.read_csv(self.data_dict['dm_path'])

        # get distance matrix as tensor
        self.dm = self.create_tensor_dm()

        self.NUM_SHIPS = len(self.fleet)
        self.NUM_PORTS = len(self.ports)
        self.NUM_DAILY_CONTRACTS = 4
        self.SET_OF_SPEEDS = [10, 12, 14]
        self.NUM_SPEEDS = len(self.SET_OF_SPEEDS)

        # the observation space changes daily based on the step == 1 day
        self.observation_space = spaces.Dict({
            "contracts": spaces.Discrete(self.NUM_DAILY_CONTRACTS),
            "ships": spaces.Discrete(self.NUM_SHIPS)
        })

        # action_space = spaces.Dict({
        #     # "choose_ship": spaces.Discrete(NUM_SHIPS+1),
        #     # we loop on every ship and take actions on each of the ships
        #     # using +1 to account for the case of not choosing a contract
        #     # for each ship we must choose:
        #     # * which contract to take among the available which are at most NUM_CONTRACTS+1
        #     # * which speed to use for the trip
        #     "choose_contract": spaces.Discrete(NUM_YEARLY_CONTRACTS+1),
        #     "choose_speed": spaces.Discrete(NUM_SPEEDS+1)
        # })

        # The action space should be described in a daily manner as well
        self.action_space = spaces.Dict({
            # we loop over the ships for the contracts of the day specified by the step
            # the actions we can take for each ship are:
            "choose_contract": spaces.Discrete(self.NUM_DAILY_CONTRACTS),
            "choose_speed": spaces.Discrete(self.NUM_SPEEDS)

        })
        self.reset()

    def step(self, action):
        """
        `step` takes a step into the environment

        Returns:
        * obs: The observation produced by the action of the agent
        * reward: The reward produced by the action of the agent
        * done: A flag signaling if the game ended
        * info : A dict useful for debugging
        """

        pass

    def reset(self, day=1):
        """
        `reset` sets the environment to its initial state

        Returns:
        * initial_state : the initial state / observation of the environment.

        """

        self.info = {}
        self.done = False

        # Set the fleet to its initial state
        self.fleet = pd.read_csv(self.data_dict['fleet_path'])

        # Calculate fleet's required cii
        self.fleet['cii_threshold'] = self.fleet['dwt'].map(cii_expected)

        # set fleet at random ports
        self.fleet['current_port'] = np.random.randint(
            1, self.NUM_PORTS+1, self.NUM_DAILY_CONTRACTS)

        # create a fleet tensor from the fleet df
        self.fleet , self.fleet_tensor = self.create_tensor_fleet()

        # Create the contracts for the first day of the year
        self.contracts, self.contracts_tensor = self.create_tensor_contracts(day)

        # update fleet tensor with ballast data
        self.fleet_tensor = self.find_ballast()

        # These contracts are passed to the mlp encoder

        # Moreover as part of the initial observation I should get the fleet tensor:

        # The fleet tensor

        # # Getting the 40 first tensor contract
        # self.idx_min, self.idx_max = 0, 40
        # self.con_first_40 = self.con_tensor[:, self.idx_min:self.idx_max, :]

        # the ships are also part of the initial state / observation

        initial_state = {"contracts": self.contracts_tensor,
                         "ships": self.fleet_tensor}

        return initial_state

    def create_contracts(self, day=1, seed=None):
        """
        `create_contracts` creats cargo contracts for a specific day of the year
        """
        con_df = pd.DataFrame(columns=['start_port_number', 'end_port_number', 'contract_type',
                              'start_day', 'end_day', 'cargo_size', 'contract_duration', 'port_distance', 'value'])

        ship_types = np.array(['supramax', 'ultramax', 'panamax', 'kamsarmax'])

        con_df['start_port_number'] = np.random.randint(
            1, self.NUM_PORTS+1, size=self.NUM_DAILY_CONTRACTS)
        con_df['contract_type'] = np.random.choice(
            ship_types, size=self.NUM_DAILY_CONTRACTS)
        con_df['end_port_number'] = np.random.randint(
            1, self.NUM_PORTS+1, size=self.NUM_DAILY_CONTRACTS)

        same_ports = con_df['start_port_number'] == con_df['end_port_number']
        # check that start and end ports are different
        while sum(same_ports) != 0:
            con_df['end_port_number'] = np.where(same_ports, np.random.randint(
                low=1, high=self.NUM_PORTS+1, size=same_ports.shape), con_df['end_port_number'])
            same_ports = con_df['start_port_number'] == con_df['end_port_number']

        con_df['start_day'] = day

        # get distance between start and end ports arrays
        start_port_numbers_index = con_df['start_port_number'] - 1
        end_port_numbers_index = con_df['end_port_number']

        dist_df = self.dm_df.iloc[start_port_numbers_index,
                                  end_port_numbers_index]

        # the distance
        con_df['port_distance'] = pd.Series(np.diag(dist_df)).reindex()

        # Create cargo size based on ship_type
        type_conditions = [con_df['contract_type'] == 'supramax',
                           con_df['contract_type'] == 'ultramax',
                           con_df['contract_type'] == 'panamax',
                           con_df['contract_type'] == 'kamsarmax']

        cargo_size_choices = [np.random.randint(40_000, 50_000, type_conditions[0].shape),
                              np.random.randint(
                                  50_000, 60_000, type_conditions[1].shape),
                              np.random.randint(
                                  60_000, 70_000, type_conditions[2].shape),
                              np.random.randint(70_000, 80_000, type_conditions[3].shape)]

        con_df['cargo_size'] = np.select(type_conditions, cargo_size_choices)

        ship_type_to_ship_code_choices = [np.ones(shape=type_conditions[0].shape),
                                          2*np.ones(shape=type_conditions[1].shape),
                                          3*np.ones(shape=type_conditions[2].shape),
                                          4*np.ones(shape=type_conditions[3].shape)]

        con_df['contract_type'] = np.select(
            type_conditions, ship_type_to_ship_code_choices)

        # calculate duration

        # pick random speed from possible set of speeds
        u_picked = np.random.choice([10, 12, 14])

        # pick distance between ports from df
        dx = con_df['port_distance']
        # find duration of trip between ports with picked speed in hours
        dt_hours = (dx / u_picked).round()
        # find duration of trip between ports in days
        dt_days = (dt_hours / 24).round()

        # get upper triangle entries of distance matrix
        x = self.dm_df.iloc[:, 1:].to_numpy(dtype=np.int32)
        mask_upper = np.triu_indices_from(x, k=1)
        triu = x[mask_upper]
        # average voyage distance between ports in the distance matrix
        avg_dx = np.round(triu.mean())
        # average voyage duration between ports with picked speed in hours
        avg_dt_hours = np.round(avg_dx/u_picked)
        # # average voyage duration between ports with picked speed in days
        avg_dt_days = np.round(avg_dt_hours / 24)

        # total duration

        con_df['contract_duration'] = dt_days + avg_dt_days

        # end_day ends at 23:59
        con_df['end_day'] = con_df['start_day'] + \
            con_df['contract_duration'] - 1

        # add contract value
        con_df['value'] = round(
            con_df['cargo_size'] * (con_df['port_distance'] / (con_df['contract_duration'] * 1_000_000)))

        return con_df

    def create_tensor_contracts(self, day):
        """
        `create_tensor_contracts` creates a tensor out of the contracts dataframe
        """
        empty = pd.DataFrame(columns=['start_port_number', 'end_port_number', 'contract_type',
                             'start_day', 'end_day', 'cargo_size', 'contract_duration', 'port_distance', 'value'])
        contracts_df = empty.copy()
        x = self.create_contracts(day)
        contracts_df = contracts_df.append(x, ignore_index=True)

        # convert everything to float for tensorflow compatibility
        contracts_df = contracts_df.astype(np.float32)

        # create the input tensor
        contracts_tensor = tf.convert_to_tensor(contracts_df)

        # add a batch size dimension
        contracts_tensor = tf.expand_dims(contracts_tensor, axis=0)

        return contracts_df, contracts_tensor

    def create_tensor_fleet(self):
        """
        `create_tensor_fleet` creates a tensor out of the fleets dataframe
        """

        # keeping only these features from the fleet df
        cols_to_keep = ['ship_number', 'dwt', 'cii_threshold', 'cii',
                        'current_port', 'current_speed', 'ship_availability', 'ballast_1', 'ballast_2', 'ballast_3', 'ballast_4']

        df = self.fleet[cols_to_keep]

        # converting to float for tensorflow compatibility
        df = df.astype(np.float32)

        # create the tensor
        tensor = tf.convert_to_tensor(df)

        # add a batch size dimension
        tensor = tf.expand_dims(tensor, axis=0)

        return df, tensor

    def create_tensor_dm(self):
        """
        `create_tensor_dm` produces a tf tensor out of the distance matrix dataframe
        Args :
        * dm_df : A dataframe containing the distance matrix data
        """
        dist_cols = self.dm_df.columns.to_list()
        del dist_cols[0]
        dm_array = self.dm_df.loc[:, dist_cols].to_numpy()
        dm_tensor = tf.convert_to_tensor(dm_array)
        return dm_tensor

    def find_distance(self, port_1_number, port_2_number):
        """
        `find_distance` returns the distance between two ports
        Args:
        * port_1_number : number of port 1
        * port_2_number : number of port 2
        """
        dist_m = self.dm
        idx_1 = port_1_number-1
        idx_2 = port_2_number-1
        distance = dist_m[idx_1, idx_2]
        return distance

    def find_ballast(self,num_contracts=4,num_ballasts=4):
        """
        `find_ballast` finds the ballast trips of fleet vessels for the new contracts
        """
        contracts = self.contracts_tensor
        fleet = self.fleet_tensor

        distance_matrix = self.dm.numpy().astype(int)

        # using -1 to get the port indeces

        # start_port idxs
        sp_idx = contracts[:,:,0] - 1
        sp_idx = tf.concat([sp_idx]*4,axis=0)

        # current_port idxs
        cp_idx = fleet[:,:,4] - 1 
        cp_idx = tf.transpose(tf.concat([cp_idx]*4,axis=0))

        # get row index 
        row_idx = cp_idx[:,1].numpy().astype(int)

        # get column index
        col_idx = sp_idx[0].numpy().astype(int)


        # get ballast data from distance matrix
        bd = distance_matrix[np.ix_(row_idx,col_idx)]

        # convert back to tf
        bd = tf.convert_to_tensor(bd)


        # duplicate bd so that dimensions are appropriate for the mask used later
        bd = tf.concat([bd]*3,axis=1)

        # casting to float and removing unwanted first column with slicing so that the dimensions align properly
        bd = tf.cast(bd[:,1:],dtype=float)

        # creating an array of ones and zeros for the mask
        oz_array = tf.concat([tf.ones([num_contracts,7]),tf.zeros([num_contracts,num_ballasts])],axis=1)

        # casting the oz_array to boolean to create the boolean mask
        mask = tf.cast(oz_array,dtype='bool')

        # populating the fleet tensor with ballast data for each contract
        fleet_with_ballast = tf.where(mask,fleet,bd)
        
        return fleet_with_ballast

        


In [42]:
from env.env import CarbonEnv
envo = CarbonEnv()

TypeError: 'int' object is not iterable

In [43]:
from gym.spaces import Box,Dict,Discrete

In [44]:
y = Discrete(3)
y

Discrete(3)

In [40]:
x = Dict({"hi":Box(0,1,shape=[3],dtype=np.int64)})
x

Dict(hi:Box(0, 1, (3,), int64))

TypeError: 'int' object is not iterable

In [ ]:
envo.create_contracts(day=1,seed=2)

In [7]:
envo.contracts

,start_port_number,end_port_number,contract_type,start_day,end_day,cargo_size,contract_duration,port_distance,value
0,5.0,10.0,2.0,1.0,4.0,53361.0,4.0,425.0,6.0
1,10.0,4.0,3.0,1.0,6.0,66063.0,6.0,1029.0,11.0
2,7.0,6.0,2.0,1.0,5.0,55810.0,5.0,502.0,6.0
3,7.0,2.0,1.0,1.0,10.0,49845.0,10.0,2503.0,12.0


In [5]:
envo.fleet

,ship_number,dwt,cii_threshold,cii,current_port,current_speed,ship_availability,ballast_1,ballast_2,ballast_3,ballast_4
0,1.0,50000.0,5.385183,0.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2.0,60000.0,4.807837,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3.0,70000.0,4.368264,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4.0,80000.0,4.020109,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [12]:
envo.fleet_tensor[:,:,-4:]

<tf.Tensor: shape=(1, 4, 4), dtype=float32, numpy=
array([[[1223.,  908.,    0.,    0.],
        [ 370.,  124.,  886.,  886.],
        [ 370.,  124.,  886.,  886.],
        [ 928.,  600.,  492.,  492.]]], dtype=float32)>

In [8]:
envo.contracts_tensor

<tf.Tensor: shape=(1, 4, 9), dtype=float32, numpy=
array([[[5.0000e+00, 1.0000e+01, 2.0000e+00, 1.0000e+00, 4.0000e+00,
         5.3361e+04, 4.0000e+00, 4.2500e+02, 6.0000e+00],
        [1.0000e+01, 4.0000e+00, 3.0000e+00, 1.0000e+00, 6.0000e+00,
         6.6063e+04, 6.0000e+00, 1.0290e+03, 1.1000e+01],
        [7.0000e+00, 6.0000e+00, 2.0000e+00, 1.0000e+00, 5.0000e+00,
         5.5810e+04, 5.0000e+00, 5.0200e+02, 6.0000e+00],
        [7.0000e+00, 2.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+01,
         4.9845e+04, 1.0000e+01, 2.5030e+03, 1.2000e+01]]], dtype=float32)>

In [ ]:
contracts = envo.create_tensor_contracts(day=1)
fleet = envo.create_tensor_fleet()


In [15]:
from env.env import CarbonEnv

envo = CarbonEnv()

TypeError: 'int' object is not iterable

In [ ]:
contracts

In [ ]:
# start_port idxs
sp_idx = contracts[:,:,0] - 1
sp_idx = tf.concat([sp_idx]*4,axis=0)

# current_port idxs
cp_idx = fleet[:,:,4] - 1 
cp_idx = tf.transpose(tf.concat([cp_idx]*4,axis=0))


In [ ]:
sp_idx

In [ ]:
cp_idx

In [ ]:
dm = envo.dm.numpy()
dm

In [ ]:
dm_tf = envo.dm
dm_tf

In [ ]:
row_idx = cp_idx[:,1].numpy().astype(int)
row_idx

In [ ]:
col_idx = sp_idx[0].numpy().astype(int)
col_idx

In [ ]:
k = dm[np.ix_(row_idx.astype(int),col_idx.astype(int))]
k = tf.convert_to_tensor(k,dtype=tf.int32)
k

In [ ]:
k = dm[np.ix_(row_idx.astype(int),col_idx.astype(int))]
k = tf.convert_to_tensor(k)
k

In [ ]:
k = tf.concat([k]*3,axis=1)
k

In [ ]:
k

In [ ]:
# # exw mia parapanw sthlh thn prwth 
# k = tf.concat([k for i in range(3)],axis=1)
# print(k.shape)


In [ ]:
# edw thn diwxnw
k = tf.cast(k[:,1:],dtype=float)
print(k.shape)

In [ ]:
oz_array = tf.concat([tf.ones([4,7]),tf.zeros([4,4])],axis=1)
mask = tf.cast(oz_array,dtype='bool')
bals = tf.where(mask,fleet,k)
bals


In [ ]:
k

In [ ]:
tf.bool(mask)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras import Input, Model
import tensorflow_probability as tfp
import numpy as np
from env.env import CarbonEnv


import numpy as np
from scipy.stats import ttest_rel
import tensorflow as tf
from tqdm import tqdm


# from data.data_functions import baseline_input_fn
# from env.enviroments import SimosFoodGroup
from models.layers import GraphAttentionEncoder,GraphAttentionDecoder

# Code version 1 

# Baseline Model
class BaselineNet():
    def __init__(self, input_size, output_size):

        self.num_contracts = 4
        self.num_ships = 10

        # Input Tensor
        contract_input = layers.Input(shape=(self.num_contracts), name="contracts")
        fleet_input = layers.Input(shape=(self.num_ships), name="fleet")


        self.model = keras.Sequential(
            layers=[
                keras.Input(shape=(input_size,)),
                layers.Dense(64, activation="relu", name="relu_layer"),
                layers.Dense(output_size, activation="linear", name="linear_layer")
            ],
            name="baseline")
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=3e-2)

    def forward(self, observations):
        output = tf.squeeze(self.model(observations))
        return output

    def update(self, observations, target):
        with tf.GradientTape() as tape:
            predictions = self.forward(observations)
            loss = tf.keras.losses.mean_squared_error(y_true=target, y_pred=predictions)
        grads = tape.gradient(loss, self.model.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))





class PolicyNet():
    def __init__(self, input_size, output_size):


        self.num_contracts = 4
        self.num_con_feats = 
        self.num_ships = 10

        contract_input = layers.Input(shape=(self.num_contracts, self.num_con_feats), name="contracts")
        fleet_input = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="fleet")
        
        # Encoding part
        initializer = tf.keras.initializers.GlorotUniform()
        x = layers.Dense(256, activation="relu", kernel_initializer=initializer, name="enc_dense_1")(contract_input)
        x = layers.Dense(64, activation="relu", kernel_initializer=initializer, name="enc_dense_2")(x)
        embedding = layers.Dense(32, activation="relu", kernel_initializer=initializer, name="enc_dense_3")(x)

        # Contracts head
        #TODO concat fleet with embedding
        x = layers.Dense(32, activation="relu", kernel_initializer=initializer,name="contract_dense_1")(embedding)
        x = layers.Dense(64, activation="relu", kernel_initializer=initializer, name="contract_dense_2")(x)
        x = layers.Dense(256, activation="relu", kernel_initializer=initializer, name="contract_dense_3")(x)
        contracts_output = layers.Dense(output_size, activation="relu", kernel_initializer=initializer, name="contract_output")(x)

        #TODO concat fleet with embedding
        x = layers.Dense(32, activation="relu", kernel_initializer=initializer,name="speed_dense_1")(embedding)
        x = layers.Dense(64, activation="relu", kernel_initializer=initializer, name="speed_dense_2")(x)
        x = layers.Dense(256, activation="relu", kernel_initializer=initializer, name="speed_dense_3")(x)
        speed_output = layers.Dense(output_size, activation="relu", kernel_initializer=initializer, name="speed_output")(x)

        model = keras.Model(inputs=[contract_input, fleet_input], 
              outputs =[contract_output, speed_output], 
              name="CarbonNetPolicy")

        self.model = keras.Sequential(
            layers=[
                keras.Input(shape=(input_size,)),
                layers.Dense(64, activation="relu", name="relu_layer"),
                layers.Dense(output_size, activation="linear", name="linear_layer")
            ],
            name="policy")

    def action_distribution(self, observations):
        logits = self.model(observations)
        return tfp.distributions.Categorical(logits=logits)

    def sample_action(self, observations):
        sampled_actions = self.action_distribution(observations).sample().numpy()
        return sampled_actions



In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input,Model
import numpy as np

from models.models import EncoderModel

class bs_net():
    def __init__(self, contract_tensor, fleet_tensor, input_size, output_size):
        

        #self.encoding_nn = EncoderModel()
        num_contracts = contract_tensor.shape[0]
        num_contract_feats = contract_tensor.shape[1]  
        num_ships  = fleet_tensor.shape[0]
        num_fleet_features = fleet_tensor.shape[1]

        contract_input = layers.Input(shape=(None, num_contracts , num_contract_feats ), name="contracts")
        fleet_input = layers.Input(shape=(None, num_ships , num_fleet_features ), name="fleet")

        self.encoding_nn = EncoderModel()


class EncoderModel():
    def __init__(self, num_contracts):
    pass

In [ ]:
CarbonEnv().create_tensor_contracts(1).shape

In [ ]:
class DecoderModel():
    pass

In [ ]:
# encoder mlp

# see https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# see https://www.tensorflow.org/tutorials/generative/autoencoder
# Create an Integer Categorical Feature for contract_type